In [1]:
import warnings
warnings.filterwarnings('ignore')

# 회원 탈퇴를 예측하는 테크닉10

#### 앞의 스포츠 센터의 데이터를 사용합니다.

#### 여기서는 use_log_months.csv와 cusomer_join.csv를 활용하겠습니다.

### 테크닉 21 : 데이터를 읽어들이고 이용데이터를 수정하자

In [2]:
import pandas as pd
customer = pd.read_csv('data/customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

#### 여기서는 과거 1개월 전의 이용 이력만으로 데이터를 작성합니다.
#### 앞에서 본 것처럼 과거 6개월을 이용할 경우, 몇 개월만에 그만 둔 회원은 예측할 수 없기 때문에 이달과 1개월 전의 이용 횟수를 집계한 데이터를 작성합니다.

In [3]:
year_months = list(uselog_months["연월"].unique()) # 컬럼 리스트화
uselog = pd.DataFrame()
for i in range(1, len(year_months)): # for문으로 과거 1개월 전 데이터들 집계
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]] #현재
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]] #한달전
    del tmp_before["연월"] #한달전에서 현재 제거
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True) #두 컬럼 합치기
uselog.head()

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### 테크닉 22 : 탈퇴전월의 탈퇴고객데이터를 작성하자

#### 왜 end_date 칼럼의 탈퇴 월이 아닌 탈퇴 전 월의 데이터를 작성할까요?
#### 탈퇴를 예측하는 목적은 탈퇴를 미연에 방지하는 것입니다. 이 스포츠 센터는 월말까지 탈퇴 신청을 해야 다음 달 말에 탈퇴할 수 있습니다. 
#### 예를 들어 2018년 8월 30일에 탈퇴 신청(탈퇴 전월)에 했다면 2018년 9월에 탈퇴 신청 완료(탈퇴일) 2018년 10월에 탈퇴를 하는 경우이기 때문에 전 월의 데이터를 사용합니다.

In [4]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None # exit_date라는 칼럼 작성
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)): # end_date의 1개월 전 계산
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str) #end_date에서 얻은 날짜를 연월로 변환, uselog를 기준으로 결합
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### exit_date 칼럼을 작성하고 end_date의 1개월 전을 계산합니다. 여기서 얻은 날짜를 연월로 변환하고 uselog를 기준으로 결합합니다.
#### 데이터 개수는 uselog를 기준으로 하기때문에 33851개가 있습니다. 결합한 데이터는 탈퇴한 회원의 탈퇴 전월의 데이터뿐임으로 결측치가 많습니다.
#### 결측치가 없는 데이터만 남기고 나머지는 제거합시다.

In [5]:
exit_uselog = exit_uselog.dropna(subset=["name"]) #결측치 제거
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


#### 지금까지 customer 데이터는 end_date 이외에 결측치는 없습니다.exit_uselog의 데이터 개수와 customer_id의 유니크한 개수를 세어보면 서로 일치하는 것을 알 수 있습니다.
#### 지금까지 탈퇴 회원의 데이터를 작성했습니다. 이제 탈퇴하지 않은 지속 회원의 데이터도 작성합시다.

### 테크닉 23 : 지속회원의 데이터를 작성하자

#### 지속 회원은 탈퇴 월이 없기 때문에 어떤 연월의 데이터를 작성해도 됩니다. 따라서 지속 회원을 추출한 후 uselog 데이터에 결합하면 작성됩니다.

In [6]:
conti_customer = customer.loc[customer["is_deleted"]==0] # 지속회원 추출
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left") #uselog와 결합
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"]) #결손 제거
print(len(conti_uselog))

33851
27422


#### name 칼럼의 결손데이터를 제거하고 탈퇴 회원을 제거합니다. 이렇게하면 데이터의 개수는 33851에서 27422개로 줄어듭니다.
#### 탈퇴 회원의 경우 1104개의 데이터만 있기때문에 27422개의 데이터를 사용하는 경우 데이터가 불균형한 경우가 발생합니다.
#### 데이터의 균형을 맞춰주기 위해서 언더샘플링 방법을 사용합니다. 

In [7]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True) # 데이터를 섞기
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id") #중복제거
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201903,PL783702,5,2.0,XXXXX,C03,F,2016-12-01,NaN,CA1,...,2_야간,6000.0,2_일반,4.166667,4.5,6.0,2.0,1.0,2019-04-30,28.0
1,201808,AS779956,4,5.0,XXX,C01,F,2016-06-01,NaN,CA1,...,0_종일,10500.0,2_일반,5.083333,5.0,7.0,3.0,1.0,2019-04-30,34.0
2,201807,HI861442,6,3.0,XXXX,C01,M,2016-07-01,NaN,CA2,...,0_종일,10500.0,0_입회비반액할인,4.416667,4.5,7.0,1.0,1.0,2019-04-30,33.0
3,201808,OA098556,7,4.0,XXXX,C02,F,2015-11-01,NaN,CA1,...,1_주간,7500.0,2_일반,4.750000,4.0,10.0,2.0,1.0,2019-04-30,41.0
4,201809,OA874597,4,2.0,XXX,C03,M,2017-02-01,NaN,CA1,...,2_야간,6000.0,2_일반,5.083333,5.0,7.0,2.0,1.0,2019-04-30,26.0


#### 언더 샘플링을 통해 데이터를 2842개까지 줄일 수 있습니다.

In [8]:
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True) #지속회원과 탈퇴 회원을 세로로 결합
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201903,PL783702,5,2.0,XXXXX,C03,F,2016-12-01,NaT,CA1,...,6000.0,2_일반,4.166667,4.5,6.0,2.0,1.0,2019-04-30,28.0,NaN
1,201808,AS779956,4,5.0,XXX,C01,F,2016-06-01,NaT,CA1,...,10500.0,2_일반,5.083333,5.0,7.0,3.0,1.0,2019-04-30,34.0,NaN
2,201807,HI861442,6,3.0,XXXX,C01,M,2016-07-01,NaT,CA2,...,10500.0,0_입회비반액할인,4.416667,4.5,7.0,1.0,1.0,2019-04-30,33.0,NaN
3,201808,OA098556,7,4.0,XXXX,C02,F,2015-11-01,NaT,CA1,...,7500.0,2_일반,4.750000,4.0,10.0,2.0,1.0,2019-04-30,41.0,NaN
4,201809,OA874597,4,2.0,XXX,C03,M,2017-02-01,NaT,CA1,...,6000.0,2_일반,5.083333,5.0,7.0,2.0,1.0,2019-04-30,26.0,NaN


#### 유지회원 2842명과 탈퇴회원 1104를 합쳐 새로운 데이터를 만듭니다.

### 테크닉 24 : 예측할 달의 재적기간을 작성하자

#### 시간적 요소가 들어간 데이터는 좋은 데이터임으로 재적 기간을 추가하는 것은 좋은 접근입니다.

In [9]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format="%Y%m") # 데이트타입 형식으로 변환
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])  # 데이트타입 형식으로 변환
for i in range(len(predict_data)): # 두 날짜의 차이를 비교하는 함수를 사용
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201903,PL783702,5,2.0,XXXXX,C03,F,2016-12-01,NaT,CA1,...,4.166667,4.5,6.0,2.0,1.0,2019-04-30,28.0,NaN,27,2019-03-01
1,201808,AS779956,4,5.0,XXX,C01,F,2016-06-01,NaT,CA1,...,5.083333,5.0,7.0,3.0,1.0,2019-04-30,34.0,NaN,26,2018-08-01
2,201807,HI861442,6,3.0,XXXX,C01,M,2016-07-01,NaT,CA2,...,4.416667,4.5,7.0,1.0,1.0,2019-04-30,33.0,NaN,24,2018-07-01
3,201808,OA098556,7,4.0,XXXX,C02,F,2015-11-01,NaT,CA1,...,4.750000,4.0,10.0,2.0,1.0,2019-04-30,41.0,NaN,33,2018-08-01
4,201809,OA874597,4,2.0,XXX,C03,M,2017-02-01,NaT,CA1,...,5.083333,5.0,7.0,2.0,1.0,2019-04-30,26.0,NaN,19,2018-09-01


### 테크닉 25 : 결측치를 제거하자

#### 앞에서도 다뤘지만 결측치가 있으면 머신러닝을 할 수 없습니다. 그래서 결측치는 제거하거나 채워야 합니다.
#### 여기서는 결측치를 제거하도록 하겠습니다.

In [10]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               244
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

#### end_date와 exit_date는 탈퇴회원을 반영한 것으로 count_1의 결측치만 제거해 줍니다.

In [11]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2650
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2650
period                  0
now_date                0
dtype: int64

### 테크닉 26 : 문자열 변수를 처리할수 있도록 가공하자

#### 가입 캠페인 구분, 회원 구분, 성별과 같은 문자열 데이터는 어떻게 처리하는 것이 좋을까요? 이 테크닉에서 자세히 알아봅시다.

In [12]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col] # 컬럼명 지정
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,2_일반,2_야간,F,2.0,1.0,27,0.0
1,2_일반,0_종일,F,5.0,1.0,26,0.0
2,0_입회비반액할인,0_종일,M,3.0,1.0,24,0.0
3,2_일반,1_주간,F,4.0,1.0,33,0.0
4,2_일반,2_야간,M,2.0,1.0,19,0.0


In [13]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,campaign_name_2_일반,class_name_0_종일,class_name_1_주간,class_name_2_야간,gender_F,gender_M
0,2.0,1.0,27,0.0,False,False,True,False,False,True,True,False
1,5.0,1.0,26,0.0,False,False,True,True,False,False,True,False
2,3.0,1.0,24,0.0,True,False,False,True,False,False,False,True
3,4.0,1.0,33,0.0,False,False,True,False,True,False,True,False
4,2.0,1.0,19,0.0,False,False,True,False,False,True,False,True


#### 판다스에서 get_dummies라는 함수를 통해 일괄적으로 더미변수를 만들 수가 있습니다. 위의 결과를 보면 좀 더 직관적으로 이해하실 수 있을겁니다.

In [14]:
del predict_data["campaign_name_2_일반"] #컬럼 삭제
del predict_data["class_name_2_야간"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
0,2.0,1.0,27,0.0,False,False,False,False,True
1,5.0,1.0,26,0.0,False,False,True,False,True
2,3.0,1.0,24,0.0,True,False,True,False,False
3,4.0,1.0,33,0.0,False,False,False,True,True
4,2.0,1.0,19,0.0,False,False,False,False,False


#### 다만 일반/종일, 주간/야간, 남자/여자같이 T/F인 경우 일부러 열을 더 늘릴 필요는 없으니 삭제시켜 줍니다.

### 테크닉 27 : 의사결정나무를 사용해서 탈퇴예측모델을 작성하자

In [ ]:
#pip install numpy==1.24.4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


#### 의사결정나무라는 모델을 적용해 분석을 시작해보겠습니다.

In [16]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit)) # 탈퇴회원과 데이터 수를 맞춰줍니다.

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"] #x에서 y삭제
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1.
 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.

#### 출력 결과에서 0은 유지, 1은 탈퇴를 의미합니다.

In [17]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
732,1.0,1.0
353,1.0,1.0
1346,0.0,0.0
493,1.0,1.0
762,1.0,1.0


#### 실행할 때마다 결과가 바뀌기 때문에 여기와 다른 결과를 보여줄 수 있습니다.

### 테크닉 28 : 예측 모델을 평가하고, 모델을 튜닝해보자

In [18]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
#result test 데이터의 y_test와 y_pred 열이 일치하는 데이터의 개수를 len으로 계산
data_count = len(results_test) #전체데이터 수
score_test = correct / data_count # 정답률 출력
print(score_test)

0.9049429657794676


#### 출력 결과를 보면 88%의 정확도를 보이는 것을 알 수 있습니다.
#### 그러나 이 데이터를 그대로 받아들이는 것은 이릅니다. 머신러닝의 목적은 미지의 데이터를 이용해서 예측하는 것이며 학습용 데이터를 활용해 예측한 것과 평가용 데이터로 예측한 것의 차이가 적은 것이 이상적입니다.

In [19]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9049429657794676
0.9790874524714829


#### 평가용 데이터의 경우 88%, 훈련용 데이터의 경우 97%로 학습용 데이터에 과적합되어있는 결과라고 볼 수 있습니다.
#### 이런 경우 데이터 늘리기, 변수 재검토, 모델의 파라미터 변경 등의 방법을 통해 이상적인 모델을 만듭니다.
#### 여기서는 모델의 파라미터 변경을 적용해 보겠습니다.

In [20]:
X = pd.concat([exit, conti], ignore_index=True) #앞의 27과 코드는 동일
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5) # 깊이만 지정
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9144486692015209
0.9252217997465145


#### 여기서 max_depth를 5로 지정해주어 의사결정나무의 깊이를 5로 제한하였습니다. 
#### 평가용과 훈련용 데이터의 격차가 크지않은 것으로 보아서 좋은 모델이 완성된 것을 알 수 있습니다.

### 테크닉 29 : 모델에 기여하고 있는 변수를 확인하자

#### 회귀와는 달리 feature_importance로 변수 중요도를 얻을 수 있습니다. 

In [21]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.348341
1,routine_flg,0.134583
2,period,0.514716
3,campaign_name_0_입회비반액할인,0.002360
4,campaign_name_1_입회비무료,0.000000
5,class_name_0_종일,0.000000
6,class_name_1_주간,0.000000
7,gender_F,0.000000


#### 결과를 보면 1개월 전의 이용 횟수, 정기 이용 여부, 재적 기간이 기여하고 있습니다.

### 테크닉 30 : 회원의 탈퇴를 예측하자

In [44]:
count_1 = 3 # 변수들을 정의
routing_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [45]:
if campaign_name == "입회비반값할인": # if문을 통해서 더미변수로 작성
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

#### category 변수를 if문으로 분기하면서 더미 변수로 작성합니다.

In [46]:
print(model.predict([input_data])) #예측 진행
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]


#### 결과를 보면 이번의 경우 탈퇴를 예측했습니다.
#### 그 다음줄의 경우 0과 1의 각각 예측 확률입니다. 여기서는 94%확률로 탈퇴라고 예측한 것 입니다. 

#### 어디까지나 최후의 판단은 인간이 하겠지만 , 예측 모델의 결과를 현명하게 활용함으로써 비즈니스를 극적으로 변화시킬 수 있습니다.